# hello_milvus Demo

    hello_milvus.ipynb demonstrates the basic operations of PyMilvus, a Python SDK of Milvus.
    Before running, make sure that you have a running Milvus instance.

1. connect to Milvus
2. create collection
3. insert data
4. create index
5. search, query, and hybrid search on entities
6. delete entities by PK
7. drop collection

In [1]:
import random
import time

from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

fmt = "\n=== {:30} ===\n"
search_latency_fmt = "search latency = {:.4f}s"

## 1. connect to Milvus

Add a new connection alias `default` for Milvus server in `localhost:19530`. Actually the "default" alias is a buildin in PyMilvus. If the address of Milvus is the same as `localhost:19530`, you can omit all
parameters and call the method as: `connections.connect()`.

Note: the `using` parameter of the following methods is default to "default".

In [2]:
connections.connect("default", host="localhost", port="19530")

has = utility.has_collection("hello_milvus")
print(f"Does collection hello_milvus exist in Milvus: {has}")

Does collection hello_milvus exist in Milvus: False


## 2. create collection
We're going to create a collection with 3 fields.

| | field name | field type | other attributes |       field description      |
|---| :--------: | :----------: | :----------------: | :----------------------------: |
|1|    "pk"    |    Int64   |  is_primary=True, auto_id=False |      "primary field"         |
|2|  "random"  |    Double  |                  |      "a double field"        |
|3|"embeddings"| FloatVector|     dim=8        |  "float vector with dim 8"   |

In [3]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]

schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")

hello_milvus = Collection("hello_milvus", schema, consistency_level="Strong")


## 3. insert data

We are going to insert 3000 rows of data into `hello_milvus`. Data to be inserted must be organized in fields.

The insert() method returns:
- either automatically generated primary keys by Milvus if auto_id=True in the schema;
- or the existing primary key field from the entities if auto_id=False in the schema.

In [4]:
entities = [
    # provide the pk field because `auto_id` is set to False
    [i for i in range(3000)],
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]

insert_result = hello_milvus.insert(entities)

print(f"Number of entities in Milvus: {hello_milvus.num_entities}")  # check the num_entites

Number of entities in Milvus: 3000


## 4. create index
We are going to create an IVF_FLAT index for hello_milvus collection.

create_index() can only be applied to `FloatVector` and `BinaryVector` fields.

In [5]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

hello_milvus.create_index("embeddings", index)

Status(code=0, message='')

## 5. search, query, and hybrid search
After data were inserted into Milvus and indexed, you can perform:
- search based on vector similarity
- query based on scalar filtering(boolean, int, etc.)
- hybrid search based on vector similarity and scalar filtering.

Before conducting a search or a query, you need to load the data in `hello_milvus` into memory.

In [6]:
hello_milvus.load()

# search based on vector similarity
print(fmt.format("Start searching based on vector similarity"))
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "l2",
    "params": {"nprobe": 10},
}

start_time = time.time()
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}, random field: {hit.entity.get('random')}")
print(search_latency_fmt.format(end_time - start_time))

# -----------------------------------------------------------------------------
# query based on scalar filtering(boolean, int, etc.)
print(fmt.format("Start querying with `random > -14`"))

start_time = time.time()
result = hello_milvus.query(expr="random > -14", output_fields=["random", "embeddings"])
end_time = time.time()

print(f"query result:\n-{result[0]}")
print(search_latency_fmt.format(end_time - start_time))

# -----------------------------------------------------------------------------
# hybrid search
print(fmt.format("Start hybrid searching with `random > -12`"))

start_time = time.time()
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > -12", output_fields=["random"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}, random field: {hit.entity.get('random')}")
print(search_latency_fmt.format(end_time - start_time))



=== Start searching based on vector similarity ===

hit: (distance: 0.0, id: 2998), random field: -20.0
hit: (distance: 0.1339951753616333, id: 1871), random field: -13.0
hit: (distance: 0.16615478694438934, id: 1180), random field: -16.0
hit: (distance: 0.0, id: 2999), random field: -16.0
hit: (distance: 0.10607236623764038, id: 764), random field: -11.0
hit: (distance: 0.14412546157836914, id: 750), random field: -11.0
search latency = 0.3159s

=== Start querying with `random > -14` ===

query result:
-{'pk': 0, 'random': -13.0, 'embeddings': [0.07525, 0.534547, 0.778204, 0.646336, 0.800183, 0.998726, 0.545411, 0.631751]}
search latency = 0.2571s

=== Start hybrid searching with `random > -12` ===

hit: (distance: 0.3116421699523926, id: 801), random field: -11.0
hit: (distance: 0.34958416223526, id: 568), random field: -11.0
hit: (distance: 0.3618723750114441, id: 1105), random field: -11.0
hit: (distance: 0.10607236623764038, id: 764), random field: -11.0
hit: (distance: 0.1441254

## 6. delete entities by PK
You can delete entities by their PK values using boolean expressions.


In [7]:
ids = insert_result.primary_keys
expr = f"pk in [{ids[2]}, {ids[3]}]"

result = hello_milvus.query(expr=expr, output_fields=["random", "embeddings"])
print(f"query before delete by expr=`{expr}` -> result: \n-{result[0]}\n-{result[1]}\n")

hello_milvus.delete(expr)

result = hello_milvus.query(expr=expr, output_fields=["random", "embeddings"])
print(f"query after delete by expr=`{expr}` -> result: {result}\n")

query before delete by expr=`pk in [2, 3]` -> result: 
-{'pk': 2, 'random': -14.0, 'embeddings': [0.976175, 0.088528, 0.806287, 0.004207, 0.30336, 0.298667, 0.279592, 0.421679]}
-{'pk': 3, 'random': -20.0, 'embeddings': [0.230225, 0.149853, 0.704977, 0.938874, 0.092708, 0.104514, 0.839864, 0.235236]}

query after delete by expr=`pk in [2, 3]` -> result: []



## 7. drop collection
Finally, drop the hello_milvus collection

In [8]:
utility.drop_collection("hello_milvus")